## Imports

In [1]:
# SQL Queries
import sqlite3

# Offline Exercise

This notebook has the questions and instructions for the offline exercise. If you haven't already, please go through the `README.md` file before you begin answering the questions here.

Make sure you do NOT commit your answers back to the repo. Instead, copy-paste this notebook together with the `touches.sqlite3` file to a separate folder in your machine, type your answers in that copied notebook, and when you're done, share the new Jupyter notebook back with the recruiter together with additional modules, graphs etc. that you may have. You would need to use the data in the `touches.sqlite3` file to answer all the questions.

### Question 1. Find the most common industry out of all the companies that were `contacted`.

In [3]:
con = sqlite3.connect("touches.sqlite3")  # Connecting SQL to database
cursor = con.cursor()  # Connecting curor/pointer to database

In [12]:
cursor.execute("""
SELECT MAX(industry) FROM companies c 
INNER JOIN persons p ON c.id = p.company_id
LEFT JOIN touches t ON p.id = t.person_id
WHERE status LIKE "completed"
""")

print(cursor.fetchall())

[('eCommerce',)]


### Code Explanation

In [13]:
"""
Question 1 wants to return the most
common industry that were contacted.

We first use the key word MAX to select
the max value of the industry column from
the table companies.

The two tables, companies and persons connect 
using an id. An inner join is performed to extract
as much information as possible.

The last statement is the WHERE statement.
Showing results where the status column has
a value of completed
"""

'\nQuestion 1 wants to return the most\ncommon industry that were contacted.\n\nWe first use the key word MAX to select\nthe max value of the industry column from\nthe table companies.\n\nThe two tables, companies and persons connect \nusing an id. An inner join is performed to extract\nas much information as possible.\n\nThe last statement is the WHERE statement.\nShowing results where the status column has\na value of completed\n'

### Question 2. What is the most common touch type sellers use when they’re making their first touch with a person? What about first touch with a company?

### Question 3. Describe the distribution of the job seniorities of people that a seller will first try to contact within a company.

To answer this question, you may use visuals, graphs, bunch of scores, tables, writeups - whatever you want. We literally want you to "describe" the distribution to us in the best way you can!

(Note: this question doesn't really have one right answer. It's more about your style of communicating the results.)

### Question 4. Describe the distribution of the mixture of job seniorities of people that a seller will touch during the entire engagement with a company.
Keep in mind that you get to decide what “mixture” means, so do begin your answer by defining it - and explaining why you think this definition makes sense. Again, feel free to use whatever visuals, graphs, bunch of scores, tables, writeups etc. that you think is appropriate for this question.

(Note: this question doesn't really have one right answer. It's more about your style of communicating the results.)

### Question 5. Build an algorithm that, when a seller is about to contact a company it never reached out to before, recommends the best/optimal mixture of job seniorities (with “mixture” as defined by you in Question 4.) they should be targeting within that company during the entire engagement.

This is the hardest question in this exercise. If you feel you have a good idea what the algorithm should look like but either feel uncomfortable coding-wise or feel pressed for time, it is perfectly OK to describe your idea in words. Be clear, explain both what it does and why you think this is the right approach.

Whether or not you go the code-route or writeup-route, be sure to explain the assumptions and choices you made, why you made them and how you would test them.

(Note: like with the previous two questions, this question doesn't really have one right answer either!)